In [1]:
from scipy.integrate import ode
from scipy.interpolate import interp1d
import pandas as pd
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook as tqdm
import time
from scipy.optimize import minimize, approx_fprime, minimize_scalar

In [2]:
import matplotlib as mpl
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['figure.autolayout'] = True 
def utkarshGrid():
    plt.minorticks_on()
    plt.grid(color='grey',
             which='minor',
             linestyle=":",
             linewidth='0.1',
             )
    plt.grid(color='black',
             which='major',
             linestyle=":",
             linewidth='0.1',
             )

In [3]:
np.set_printoptions(precision=2)

In [4]:
G = 6.67259e-8 # dyne cm2 g-2
c = 2.99792458e10 # cms-1
n_den = 2.3e14
km2cm = 1e5
msun = 1.98847e33

def lamba_metric(M, R):
    return -np.log((1-2*G*M/(c**2*R)))

def df_to_ep(df):
    e_den = df.energy_densityc2
    p_arr = df.pressurec2
    p = p_arr[p_arr > 1e8]
    e_den = e_den[p_arr > 1e8]
    e_den = e_den*((2.99792458e10)**2)
    pressure = p*((2.99792458e10)**2)
    return e_den.to_numpy(), pressure.to_numpy()
def get_ep(e, p):
    f_e_smooth = interp1d(p, e, fill_value= "extrapolate", kind = "cubic")
    return f_e_smooth
def get_pe(p, e):
    f_e_smooth = interp1d(e, p, fill_value=(0, 0), kind = "cubic", bounds_error=True)
    return f_e_smooth


path = "2205.02081/SLY4.csv"
df = pd.read_csv(path)
e, p = df_to_ep(df)
p_max = max(p)
p_min = max(1*(c**2), min(p)) 
EOS = get_ep(e, p)

In [5]:
def _dMdr(r, e):
    return 4 * np.pi * r**2 * (e/(c**2))
def _b(r, M):
    return (G*M)/((c**2)*r)
def _dnudr(r, Q, lamda):
    return (2/r) * np.exp(lamda) * Q
def _Q(r, P, M):
    frac = (4 * np.pi * G * (r**2)*P)/(c**4)
    return _b(r, M) + frac
def _dPdr(r, P, M, e):    
    num = (M+4*np.pi*(r**3)*P/(c**2))
    dem = r * (r-2*G*M/(c**2))
    return -1 * (e+P) * G/(c**2) * num/dem
def dedP_helper(p, e):
    return np.gradient(e, p), e
def dedP(p, e):
    dedp_helper, e_arr = dedP_helper(p, e)
    return interp1d(e_arr, dedp_helper, fill_value="extrapolate", kind = "cubic")
def drhodP(e, dedp = dedP(p, e)):
    return (c**-2) * dedp(e)

In [6]:
# # DIMENSIONS CHECKED
# def _H0(r, nu, X, nl, Q, omega, lamda, H1, b, K):
#     term1 = 8 * np.pi * (r**2) * np.exp(-nu/2)* X * (G/(c**4))
#     term2 = -H1 * ((nl+1)*Q - (((omega*r)**2)/(c**2))*np.exp(-(nu+lamda)))
#     term3 = K * (nl - ((((omega*r)**2)/(c**2)) * np.exp(-nu) - Q*(np.exp(lamda)*Q - 1)))
#     factor = (2*b + nl + Q)**(-1)
#     return factor * (term1 + term2 + term3)

# # DIMENSIONS CHECKED
# def _V(r, X, e, p, Q, nu, lamda, W, H0, omega):
#     factor = (c**2)*np.exp(nu/2)/(omega**2)
#     term1 = X/(e+p)
#     term2 = - Q/(r**2)*np.exp(nu + lamda)*W
#     term3 = - np.exp(nu/2)*H0/2
#     term = term1 + term2 + term3
#     return factor * term

# # DIMENSIONS CHECKED
# def _dH1dr(r, l, b, lamda, p, e, H1, H0, K, V):
#     term1 = -H1 * (l+1+2*b*np.exp(lamda) + 4*np.pi*(r**2)*np.exp(lamda)*(p-e)*(G/(c**4)))
#     term2 = np.exp(lamda) * (H0 + K - 16*np.pi*(e+p)*V*(G/(c**4)))
#     return 1/r* (term1 + term2)

# # DIMENSIONS CHECKED
# def _dKdr(r, H0, nl, H1, lamda, Q, l, K, e, p, W):
#     term1= H0
#     term2= (nl+1)*H1
#     term3= (np.exp(lamda)*Q - l - 1)*K
#     term4= -8*np.pi*(e+p)*np.exp(lamda/2)*W*(G/(c**4))
#     return 1/r * (term1 + term2 + term3 + term4)

# # DIMENSIONS CHECKED
# def _dWdR(r, W, l, lamda, V, e, p, X, cad2_inv, H0, K, nu):
#     term1 = -1*(l+1)*(W+l*np.exp(lamda/2)*V)
#     term2_1 = (np.exp(-nu/2)*X*(cad2_inv/(e+p))*(c**2))
#     term2_2 = H0/2 * 1
#     term2_3 = K * 1
#     term2_factor = (r**2)*np.exp(lamda/2)
#     term2 = term2_factor*(term2_1 + term2_2 + term2_3)
#     return 1/r * (term1 + term2)

# # DIMENSIONS CHECKED
# def _dXdr(r, l, X, e, p, nu, lamda, Q, H0, omega, nl, H1, K, V, W, derivative_term):
#     term1= -l*X
#     term2_factor= (e+p)*np.exp(nu/2)/2 
#     term2_1= H0 * (1-np.exp(lamda)*Q)
#     term2_2= H1 * ((((omega*r)**2)/(c**2))*np.exp(-nu) + nl + 1)
#     term2_3= K * (3 * np.exp(lamda)*Q - 1)
#     term2_4= (4 * (nl+1)*np.exp(lamda)*Q*V/(r**2))
#     term2_5_factor= -2*W
#     term2_5_1= ((omega**2)/(c**2))*np.exp(lamda/2 - nu)
#     term2_5_2= 4*np.pi*(e+p)*np.exp(lamda/2)*(G/(c**4))
#     term2_5_3= -1*(r**2) * derivative_term
#     term2_5= term2_5_factor * (term2_5_1 + term2_5_2 + term2_5_3)
#     term2 = term2_factor * (term2_1 + term2_2 + term2_3 + term2_4 + term2_5)
#     return 1/r * (term1 + term2)

In [7]:
# DIMENSIONS CHECKED
def _H0(r, nu, X, nl, Q, omega, lamda, H1, b, K):
    omgr2_c2 = ((omega * r)**2)/(c**2)
    factor = (2*b + nl + Q) ** (-1)
    term1 = 8 * np.pi * (r**2) * np.exp(-nu/2) * X * (G/(c**4)) * H1 * (Q*(nl+1) - omgr2_c2 * np.exp(-nu-lamda))
    term2 = K * (nl - omgr2_c2*np.exp(-nu)-Q*(np.exp(lamda)*Q-1))
    return factor * (term1 + term2)

# DIMENSIONS CHECKED
def _V(r, X, e, p, Q, nu, lamda, W, H0, omega):
    factor = np.exp(nu/2) * (c**2)/(omega**2)
    term1 = X/(e+p)
    term2 = -Q/(r**2) * W * np.exp((nu+lamda)/2)
    term3 = -np.exp(nu/2)*H0/2
    return factor * (term1 + term2 + term3)

In [8]:
# DIMENSIONS CHECKED
def _dH1dr(r, l, b, lamda, p, e, H1, H0, K, V):
    G_c4 = G/(c**4)
    term1 = -H1*(l + 1 + 2*b*np.exp(lamda) + G_c4 * 4*np.pi*(r**2)*np.exp(lamda)*(p-e))
    term2 = np.exp(lamda) * (H0 + K - G_c4*16*np.pi*(e+p)*V)
    return (1/r) * (term1 + term2)

# DIMENSIONS CHECKED
def _dKdr(r, H0, nl, H1, lamda, Q, l, K, e, p, W):
    term1 = H0
    term2 = (nl+1)*H1
    term3 = (np.exp(lamda)*Q-l-1)*K
    term4 = -8*np.pi*(e+p)*np.exp(lamda/2)*W*G/(c**4)
    return (1/r) * (term1 + term2 + term3 + term4)

# DIMENSIONS CHECKED
def _dWdR(r, W, l, lamda, V, e, p, X, cad2_inv, H0, K, nu):
    term1 = -(l+1)*(W+l*np.exp(lamda/2)*V)
    term2_fac = (r**2 * np.exp(lamda/2))
    term2_1 = (np.exp(-nu/2)*X*(c**2) * cad2_inv)/(e+p)
    term2_2 = H0/2
    term2_3 = K
    term2 = term2_fac * (term2_1 + term2_2 + term2_3)
    return (1/r) * (term1 + term2)

# DIMENSIONS CHECKED
def _dXdr(r, l, X, e, p, nu, lamda, Q, H0, omega, nl, H1, K, V, W, derivative_term):
    omgr2_c2 = ((omega * r)**2)/(c**2)
    G_c4 = G/(c**4)
    term1 = -l*X
    term2_fac = (e+p)*np.exp(nu/2)/2
    term2_1 = (1-np.exp(lamda)*Q)*H0
    term2_2 = H1 * (omgr2_c2*np.exp(-nu) + nl + 1)
    term2_3 = K * (3*np.exp(lamda)*Q - 1)
    term2_4 = 4 * (nl + 1) * np.exp(lamda) * Q * V/(r**2)
    term2_5_fac = -2*W
    term2_5_1 = np.exp(lamda/2 - nu)*(omega**2)/(c**2)
    term2_5_2 = G_c4 * 4 * np.pi * (e+p) * np.exp(lamda/2)
    term2_5_3 = -(r**2) * derivative_term
    term2_5 =  term2_5_fac * (term2_5_1 + term2_5_2 + term2_5_3)
    term2 = term2_fac * (term2_1 + term2_2 + term2_3 + term2_4 + term2_5)
    return (1/r) * (term1 + term2)

In [9]:
def coupledTOV(r, VEC, init_params):
    P, M, nu, H1, K, W, X = VEC
    EOS, l, omega, p_min, p_max, nl = init_params
    if P <= p_min: 
        return None
    if P >= p_max:
        return None
    if 2*_b(r, M) >= 1:
        return None
    b = _b(r, M)
    lamda = np.log(1/(1-2*b))
    Q = _Q(r, P, M)
    e = EOS(P) 
    cad2_inv = drhodP(e)
    derivative_term = 0 # CHANGE THIS ONCE YOU GET CORRECT MASS/RADIUS RELATIONSHIP
    
    dPdr = _dPdr(r, P, M, e)
    dMdr = _dMdr(r, e)
    dnudr = _dnudr(r, Q, lamda)
    H0 = _H0(r, nu, X, nl, Q, omega, lamda, H1, b, K)
    V = _V(r, X, e, P, Q, nu, lamda, W, H0, omega)
    dH1dr = _dH1dr(r, l, b, lamda, P, e, H1, H0, K, V)
    dKdr = _dKdr(r, H0, nl, H1, lamda, Q, l, K, e, P, W)
    dWdr = _dWdR(r, W, l, lamda, V, e, P, X, cad2_inv, H0, K, nu)
    dXdr = _dXdr(r, l, X, e, P, nu, lamda, Q, H0, omega, nl, H1, K, V, W, derivative_term)
    ret = np.array([dPdr, dMdr, dnudr, dH1dr, dKdr, dWdr, dXdr])
    print(ret)
    return ret

In [10]:
def tov(EOS, init_VEC, r_i, p_min,p_max, omega,nl, progress = False, 
        l=2, n_iter_max = 20000):
    init_params = [EOS, l, omega, p_min, p_max, nl]
    r = ode(lambda r, VEC: coupledTOV(r, VEC, init_params)).set_integrator('VODE')
    r.set_initial_value(init_VEC, r_i)
    results = []
    r_list = []
    i = 0
    r_max = 20 * km2cm
    max_iter = n_iter_max
    dr = r_max/max_iter
    if progress:
        pbar = tqdm(total=max_iter)
    while r.successful() and (r.y[0] >= p_min):
        i+=1
        integral = r.integrate(r.t+dr)
#         print(r.successful())
        if progress:
            pbar.update(1)
        if i > max_iter:
            print("[STATUS] max_iter reached")
            break
        if (r.y[0] < p_min):
            break
        if not r.successful():
            break
        results.append(integral)
        r_list.append(r.t+dr)
    if progress:
        pbar.close()
    
#     print("Integration, Pressure: ", r.successful(), (r.y[0] >= p_min))
    
    results = np.array(results, dtype = float)
    p, m, nu, h1, k, w, x = results.T
    r = np.array(r_list)
    return p, m, r, nu, h1, k, w, x

In [11]:
def initial_conditions(EOS, e, p, k, km2cm = 1e5, r_i = 1):
    p0 = p[k]
    e0 = EOS(p0)
    p_c = p0 - 2 * np.pi * (G/(c**4)) * r_i**2 * (p0 + e0) * (3*p0 + e0)/3
    e_c = EOS(p_c)
    m0 = e_c/(c**2) * 4/3 * np.pi * r_i**3
    omega = 2*(2*np.pi) #Arbitrary guess
    l=2 
    nl = (l-1)*(l+2)/2
    nu0 = -1 
    Q0 = _Q(r_i, p_c, m0)
    b0 = _b(r_i, m0)
    W0 = 1
    
#     K0 = 0.5 * (e_c + p_c) * (G/(c**2)) # Only thing im guessing right now other than nu0
    K0 = -1
    
    # DIMENSIONS CHECKED
    X0_factor = (e_c + p_c) * np.exp(nu0/2)
    X0_term1 = 4 * np.pi/3 * (e_c + 3*p_c) * W0 * G/(c**4)
    X0_term2 = -(omega**2)/l * np.exp(-nu0) * W0  /(c**2)
    X0_term3 = K0/2
    X0 = X0_factor * (X0_term1 + X0_term2 + X0_term3)
    H00 = K0
    H10 = (l*K0 + 8*np.pi*(G/(c**4))*(e_c+p_c)*W0)/((nl+1))
    
    init_VEC = [p_c, m0, nu0, H10, K0, W0, X0]
    p_max = max(p)
    p_min = max(c**2, min(p)) 
    print("Initial: ", init_VEC)
    return p_c, e_c, m0, omega, l, nl, nu0, Q0, b0, W0, K0, X0, H00, H10, init_VEC, \
    p_max, p_min, r_i

In [12]:
def surface_conditions(p, m, r_arr, nu, h1, k, w, x):
    max_idx = np.argmax(m) - 1
    m_R = m.max() # In units of msun
    r_R = r_arr[max_idx] # In units of km
    p_R = p[max_idx] #cgs
    ec_R = EOS(p_R) #cgs
    nu_R = nu[max_idx]
    h1_R = h1[max_idx]
    k_R = k[max_idx]
    w_R = w[max_idx]
    x_R = x[max_idx]
    schild = (1-2*G*m_R/(c**2*r_R))
    interior = np.exp(nu_R)
    return max_idx, m_R, r_R, p_R, ec_R, nu_R, h1_R, k_R, w_R, x_R, schild, interior

def print_params(p, m, r_arr, nu, h1, k, w, x):
    max_idx, m_R, r_R, p_R, ec_R, nu_R, h1_R, k_R, w_R, x_R, schild, \
    interior = surface_conditions(p, m, r_arr, nu, h1, k, w, x)
    print(f"Star has mass {m_R/msun:.3f} Msun and radius {r_R/km2cm:.3f}km")
    print(f"Interior Surface: {interior:.8f}")
    print(f"Exterior Surface: {schild:.8f}")
    print(f"v0: {nu0}")
    print(f"Lamda: {lamba_metric(m_R, r_R)}")
    print(f"Boundary Term: {x_R}")
    return None

In [13]:
p_c, e_c, m0, omega, l, nl, nu0, Q0, b0, W0, K0, X0, H00, H10, init_VEC, \
p_max, p_min, r_i = initial_conditions(EOS, e, p, -92)

p, m, r_arr, nu, h1, k, w, x = tov(EOS, init_VEC, r_i, p_min,p_max, 
                           omega, nl, progress = True, l = l)

max_idx, m_R, r_R, p_R, ec_R, nu_R, h1_R, k_R, w_R, x_R, schild, \
interior = surface_conditions(p, m, r_arr, nu, h1, k, w, x)

print("== First Integration == ")
print_params(p, m, r_arr, nu, h1, k, w, x)

# Update init_VEC
nu_ext = -lamba_metric(m_R, r_R)
nu_int = nu_R #At surface
delta_nu = nu_int - nu_ext
nu0 =  nu0 - delta_nu

init_VEC = [p_c, m0, nu0, H10, K0, W0, X0]
time.sleep(0.2)

p, m, r_arr, nu, h1, k, w, x = tov(EOS, init_VEC, r_i, p_min,p_max, 
                           omega, nl, progress = True, l = l)

max_idx, m_R, r_R, p_R, ec_R, nu_R, h1_R, k_R, w_R, x_R, schild, \
interior = surface_conditions(p, m, r_arr, nu, h1, k, w, x)

print("== Corrected Integration == ")
print_params(p, m, r_arr, nu, h1, k, w, x)

Initial:  [1.2999963394850814e+35, 4054675614401985.5, -1, -0.6666666666659746, -1, 1, -3.0325893664368066e+35]


[-4.36e+23  1.22e+16  8.72e-13  1.31e-06 -1.35e-13  1.89e+06 -5.00e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.31e-06 -1.35e-13  1.90e+06 -5.02e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.31e-06 -1.35e-13  1.90e+06 -5.02e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.31e-06 -1.35e-13  1.90e+06 -5.02e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.32e-06 -1.35e-13  1.90e+06 -5.03e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.32e-06 -1.35e-13  1.90e+06 -5.03e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.35e-06 -1.35e-13  1.96e+06 -5.17e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.35e-06 -1.35e-13  1.96e+06 -5.17e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.33e-06 -1.35e-13  1.92e+06 -5.07e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.33e-06 -1.35e-13  1.92e+06 -5.07e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.34e-06 -1.35e-13  1.93e+06 -5.11e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.44e-06 -1.35e-13  2.09e+06 -5.52e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.44e-06 -1.35e-13  2.09e+06 -5.52e+29]
[-4.36e+23  1.22e+16  8.72e-13  1.38e-06 -1.35e-13  1.99e+06 -5.

[-4.36e+23  1.22e+16  8.72e-13  2.82e+05 -1.34e-13  4.07e+17 -1.08e+41]
[-4.36e+23  1.22e+16  8.72e-13  2.82e+05 -1.34e-13  4.07e+17 -1.08e+41]
[-4.36e+23  1.22e+16  8.72e-13  3.29e+05 -1.34e-13  4.75e+17 -1.26e+41]
[-4.36e+23  1.22e+16  8.72e-13  3.29e+05 -1.34e-13  4.75e+17 -1.26e+41]
[-4.36e+23  1.22e+16  8.72e-13  3.84e+05 -1.34e-13  5.54e+17 -1.47e+41]
[-4.36e+23  1.22e+16  8.72e-13  3.84e+05 -1.34e-13  5.54e+17 -1.47e+41]
[-4.36e+23  1.22e+16  8.72e-13  4.48e+05 -1.34e-13  6.47e+17 -1.71e+41]
[-4.36e+23  1.22e+16  8.72e-13  4.48e+05 -1.34e-13  6.47e+17 -1.71e+41]
[-4.36e+23  1.22e+16  8.72e-13  5.22e+05 -1.34e-13  7.55e+17 -2.00e+41]
[-4.36e+23  1.22e+16  8.72e-13  5.22e+05 -1.34e-13  7.55e+17 -2.00e+41]
[-4.36e+23  1.22e+16  8.72e-13  6.10e+05 -1.34e-13  8.81e+17 -2.33e+41]
[-4.36e+23  1.22e+16  8.72e-13  6.10e+05 -1.34e-13  8.81e+17 -2.33e+41]
[-4.36e+23  1.22e+16  8.72e-13  7.12e+05 -1.34e-13  1.03e+18 -2.72e+41]
[-4.36e+23  1.22e+16  8.72e-13  7.12e+05 -1.34e-13  1.03e+18 -2.

[-4.36e+23  1.22e+16  8.72e-13  4.32e+11  2.91e-11  6.25e+23 -1.65e+47]
[-4.36e+23  1.22e+16  8.72e-13  4.32e+11  1.46e-11  6.25e+23 -1.65e+47]
[-4.36e+23  1.22e+16  8.72e-13  5.17e+11  2.91e-11  7.46e+23 -1.97e+47]
[-4.36e+23  1.22e+16  8.72e-13  5.17e+11  5.82e-11  7.46e+23 -1.97e+47]
[-4.36e+23  1.22e+16  8.72e-13  6.17e+11  2.91e-11  8.92e+23 -2.36e+47]
[-4.36e+23  1.22e+16  8.72e-13  6.17e+11  2.91e-11  8.92e+23 -2.36e+47]
[-4.36e+23  1.22e+16  8.72e-13  7.37e+11  2.91e-11  1.07e+24 -2.82e+47]
[-4.36e+23  1.22e+16  8.72e-13  7.37e+11  2.91e-11  1.07e+24 -2.82e+47]
[-4.36e+23  1.22e+16  8.72e-13  8.80e+11 -5.82e-11  1.27e+24 -3.36e+47]
[-4.36e+23  1.22e+16  8.72e-13  8.80e+11  0.00e+00  1.27e+24 -3.36e+47]
[-4.36e+23  1.22e+16  8.72e-13  1.05e+12 -5.82e-11  1.52e+24 -4.02e+47]
[-4.36e+23  1.22e+16  8.72e-13  1.05e+12 -5.82e-11  1.52e+24 -4.02e+47]
[-4.36e+23  1.22e+16  8.72e-13  1.26e+12  0.00e+00  1.82e+24 -4.80e+47]
[-4.36e+23  1.22e+16  8.72e-13  1.26e+12  5.82e-11  1.82e+24 -4.

[-4.36e+23  1.22e+16  8.72e-13  2.08e+23 -3.10e+08  3.01e+35 -7.96e+58]
[-4.36e+23  1.22e+16  8.72e-13  2.29e+23 -3.73e+08  3.30e+35 -8.74e+58]
[-4.36e+23  1.22e+16  8.72e-13  2.29e+23 -3.73e+08  3.30e+35 -8.74e+58]
[-4.36e+23  1.22e+16  8.72e-13  2.51e+23 -4.49e+08  3.63e+35 -9.59e+58]
[-4.36e+23  1.22e+16  8.72e-13  2.51e+23 -4.49e+08  3.63e+35 -9.59e+58]
[-4.36e+23  1.22e+16  8.72e-13  2.75e+23 -5.40e+08  3.98e+35 -1.05e+59]
[-4.36e+23  1.22e+16  8.72e-13  2.75e+23 -5.40e+08  3.98e+35 -1.05e+59]
[-4.36e+23  1.22e+16  8.72e-13  3.02e+23 -6.51e+08  4.37e+35 -1.15e+59]
[-4.36e+23  1.22e+16  8.72e-13  3.02e+23 -6.51e+08  4.37e+35 -1.15e+59]
[-4.36e+23  1.22e+16  8.72e-13  3.32e+23 -7.84e+08  4.79e+35 -1.27e+59]
[-4.36e+23  1.22e+16  8.72e-13  3.32e+23 -7.84e+08  4.79e+35 -1.27e+59]
[-4.36e+23  1.22e+16  8.72e-13  3.64e+23 -9.44e+08  5.26e+35 -1.39e+59]
[-4.36e+23  1.22e+16  8.72e-13  3.64e+23 -9.44e+08  5.26e+35 -1.39e+59]
[-4.36e+23  1.22e+16  8.72e-13  3.99e+23 -1.14e+09  5.77e+35 -1.

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/integrate/_ode.py:1011: UserWarning: vode: Excess work done on this call. (Perhaps wrong MF.)
  warnings.warn('{:s}: {:s}'.format(self.__class__.__name__,


ValueError: not enough values to unpack (expected 7, got 0)

In [ ]:
p, m, r_arr, nu, h1, k, w, x

plt.figure()
plt.plot(r_arr/km2cm, p/p_c)
plt.xlabel("r")
plt.ylabel("P/Pc")

plt.figure()
plt.plot(r_arr/km2cm, m/msun)
plt.xlabel("r ")
plt.ylabel("M/Msun")
plt.show()

plt.figure()
plt.plot(r_arr/km2cm, nu)
plt.xlabel("r ")
plt.ylabel("v")
plt.show()

plt.figure()
plt.plot(r_arr/km2cm, h1)
plt.xlabel("r ")
plt.ylabel("H1")
plt.autoscale()
plt.show()

plt.figure()
plt.plot(r_arr/km2cm, k)
plt.xlabel("r ")
plt.ylabel("K")
plt.autoscale()
plt.show()

plt.figure()
plt.plot(r_arr/km2cm, w)
plt.xlabel("r ")
plt.ylabel("W")
plt.autoscale()
plt.show()

plt.figure()
plt.plot(r_arr/km2cm, x)
plt.xlabel("r ")
plt.ylabel("X")
plt.autoscale()
plt.show()